***IMPORTS Y ACCESO A DATOS DE CSV BICIS***

In [146]:
#Importación de paquetes y lectura de csv:
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import random
from sklearn import model_selection
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr
import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import copy

random.seed(27)
dataBicis = pd.read_csv('hour.csv')
dataBicis = dataBicis.drop("dteday", axis=1)
dataBicis = dataBicis.drop("instant", axis=1)
atributosBicis = dataBicis.loc[:, 'season':'registered']
objetivoBicis = dataBicis['cnt']

(atributos_entrenamientoBicis, atributos_pruebaBicis,
 objetivo_entrenamientoBicis, objetivo_pruebaBicis) = model_selection.train_test_split(
    atributosBicis, objetivoBicis, test_size=.33)

atributos_entrenamientoBicis = atributos_entrenamientoBicis.values
objetivo_entrenamientoBicis = objetivo_entrenamientoBicis.values
atributos_pruebaBicis = atributos_pruebaBicis.values
objetivo_pruebaBicis= objetivo_pruebaBicis.values

rangosBicis = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in dataBicis.columns:
    valor_min = dataBicis[columna].min()
    valor_max = dataBicis[columna].max()
    rangosBicis[columna] = {'min': valor_min, 'max': valor_max}

***MÉTODO LIME***

In [147]:
def metodo_lime(N,x,k,model,rangos):
    random.seed(1)
    Xprima=[]
    R=[]
    W=[]
    Yprima= []
    for i in range (0,N):
        atributosAPerturbar=seleccionar_atributos_aleatorios(x,k)
        xprima = perturbar_muestra(x,atributosAPerturbar,rangos)
        w = distancia_entre_muestras(x,xprima)
        r=atributosAPerturbar
        Xprima.append(xprima)
        R.append(r)
        W.append(w)
        yprima = prediccion_de_modelo(xprima,model)
        Yprima.append(yprima)
    W=np.array(W)
    W = W.reshape(W.shape[0])
    G = entrenar_modelo_ridge(R,Yprima,W)
    return G.coef_

#Se seleccional los k atributos aleatorios a modificar

def seleccionar_atributos_aleatorios(dataframe, k):
    columnas = dataframe.columns.tolist()
    seleccionados = random.sample(range(len(columnas)), k)
    lista = [0] * len(columnas)
    for i in seleccionados:
        lista[i] = 1
    #print(lista)
    return lista

#Se perturba la muestra original de forma que los atributos seleccionados pueden variar en un rango en función del valor máximo y mínimo de dicho atributo
def perturbar_muestra(x, atributosAPerturbar,rangos):
    xprima = x.copy()
    for i in range(0,len(atributosAPerturbar)):
        if atributosAPerturbar[i] == 1:
            rango_min = xprima[i] - rangos[list(rangos.keys())[i]]['min']
            rango_max = rangos[list(rangos.keys())[i]]['max'] - xprima[i]
            xprima[i] = random.uniform(rango_min, rango_max)
    return xprima

# Se calcula la distancia coseno entre la muestra original y la perturbada
def distancia_entre_muestras(x,xprima):
    return cosine_distances(x.values, xprima.values)

#Se predice el resultado de la muestra con el modelo elegido
def prediccion_de_modelo(xprima,model):
    if not isinstance(model, (RandomForestRegressor, RandomForestClassifier)):
        xprima = DMatrix(xprima)
    y_pred = model.predict(xprima)
    return y_pred


#Se entrena el modelo de regresión Ridge con las muestras perturbadas y los pesos calculados
def entrenar_modelo_ridge(R, Yprima, W):
    modelo_entrenado = Ridge(alpha=1.0) 
    modelo_entrenado.fit(R, Yprima, sample_weight=W)
    return modelo_entrenado

***MODELOS BICIS***

In [148]:
#RandomForest
#Creamos el modelo RandomForest (En este caso es Regressor ya que se trata de una tarea de regresión)
modelRandomForestBicis = RandomForestRegressor(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestBicis.fit(atributos_entrenamientoBicis, objetivo_entrenamientoBicis)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainBicis = xgb.DMatrix(atributos_entrenamientoBicis, label=objetivo_entrenamientoBicis)
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestBicis2 = xgb.DMatrix(atributos_pruebaBicis)
# Definimos los parámetros del modelo XGBoost
paramsBicis = {
    'objective': 'reg:squarederror',  # Objetivo para regresión
    'eval_metric': 'rmse'  # Métrica de evaluación, error cuadrático medio (RMSE)
}

# Entrenamos el modelo XGBoost
modelXGBoostBicis = xgb.train(paramsBicis, dtrainBicis)

In [149]:
clon_model_XGboost_bicis = copy.deepcopy(modelXGBoostBicis)
prediccionesBicisXG = clon_model_XGboost_bicis.predict(dtestBicis)
explicacionesBicisXG = []
atributosAcambiar = 5
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_bicis,rangosBicis)
    explicacionesBicisXG.append(explicacion)
print(explicacionesBicisXG)

[array([[-5.96622792e+00, -1.55754281e+01, -1.85672235e+00,
        -1.80094864e+00, -4.16606178e-01, -4.27978872e+01,
         2.18426418e+01,  2.28119677e+01, -7.70829537e+00,
        -3.80680285e-01, -1.51588219e+01,  1.19603714e-02,
         2.29255216e+00,  4.47024958e+01]]), array([[ 1.55334181, -0.50854294, -0.01432419, -0.01448736, -0.0032682 ,
         1.68908509, -0.74884994, -0.92591097,  1.13125714,  0.29110454,
        -0.50527474, -0.29344515,  0.02314821, -1.67383332]]), array([[-1.3762081 , -0.03375651, -0.01558346, -0.80574796, -0.00194611,
        -0.12894476,  0.78352565, -0.63899743,  0.29942995, -1.41632507,
        -0.0318104 ,  1.41646427,  1.80328642,  0.14661352]]), array([[ -5.17030655, -18.15789018,  -3.11893557,  -1.51776622,
         -0.51477483, -51.78909426,  32.35846321,  22.54956661,
        -10.92655987,   9.90145456, -17.64311535, -10.26644519,
         -0.76241039,  55.05781403]]), array([[ -56.73146259,  -52.57539973,  -16.64489272, -108.64392126,
 

In [150]:
prediccionesBicisRF = modelRandomForestBicis.predict(atributos_pruebaBicis)
explicacionesBicisRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestBicis,rangosBicis)
    explicacionesBicisRF.append(explicacion)
print(explicacionesBicisRF)

[array([[ -4.06668573, -16.34477156,  -2.31851364,  -2.3463486 ,
         -0.50089534, -35.7167066 ,  17.84982824,  20.18539201,
         -1.64355066,  -3.47350137, -15.84387622,   3.03148269,
          3.09404989,  38.09409691]]), array([[ 0.81069687, -0.24597523, -0.01093442, -0.01145769, -0.00248185,
         1.32260187, -0.4295703 , -0.88209715,  0.98767229,  0.1309509 ,
        -0.24349337, -0.13278948,  0.01790176, -1.31102418]]), array([[-0.99441984, -0.03287102, -0.0150456 , -0.78809321, -0.00187554,
        -0.12823591,  0.53221778, -0.38893627,  0.09138766, -1.26789909,
        -0.03099549,  1.26800989,  1.61148879,  0.14526785]]), array([[-5.20956459e+00, -1.88094303e+01, -3.26587753e+00,
        -1.73028928e+00, -5.40598303e-01, -5.32357448e+01,
         3.37565334e+01,  2.27450889e+01, -1.17131304e+01,
         1.05065177e+01, -1.82688320e+01, -1.08898222e+01,
        -3.76668667e-03,  5.66589161e+01]]), array([[ -54.54083755,  -57.99944939,  -16.71208996, -104.76109719,
 

***VALIDACIÓN DE MODELO***

In [151]:
def evaluate_model(model, Xentrenamiento, yentrenamiento, Xtest, ytest):
    # Validación cruzada
    cv_scores = cross_val_score(model, Xentrenamiento, yentrenamiento, cv=2)
    mean_cv_score = np.mean(cv_scores)
    
    print("Puntuación media de validación cruzada:", mean_cv_score)
    
    #.score
    score = model.score(Xtest, ytest)
    print("Puntuación en el conjunto de prueba:", score)

In [152]:
evaluate_model(modelRandomForestBicis, atributos_entrenamientoBicis, objetivo_entrenamientoBicis, atributos_pruebaBicis, objetivo_pruebaBicis)

Puntuación media de validación cruzada: 0.9994650191342412
Puntuación en el conjunto de prueba: 0.9997180291963875


******

In [153]:


nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoBicis).iloc[3]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)


Iterac=10
atributosAcambiar=5

nuevos_ejemplos = pd.DataFrame([[1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7]])
#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestBicis))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostBicis))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestBicis,rangosBicis))

#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostBicis,rangosBicis))

    0    1    2     3    4    5    6    7     8       9     10      11   12  \
3  1.0  1.0  2.0  21.0  0.0  5.0  1.0  1.0  0.34  0.3485  0.49  0.1045  8.0   

      13  
3  126.0  


******

***

***ACCESO A DATOS DE CSV PEN***

In [154]:

# Cargar el archivo CSV
pen = pd.read_csv('pendigits.csv', header=0, names=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15','a16',"class-objective"])

atributosPen = pen.loc[:, 'a1':'a16']
objetivoPen = pen['class-objective']

rangosPen = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in pen.columns:
    valor_min = pen[columna].min()
    valor_max = pen[columna].max()
    rangosPen[columna] = {'min': valor_min, 'max': valor_max}
    


In [155]:
(atributos_entrenamientoPen, atributos_pruebaPen,
 objetivo_entrenamientoPen, objetivo_pruebaPen) = model_selection.train_test_split(
    atributosPen, objetivoPen, test_size=.33,stratify=objetivoPen)

atributos_entrenamientoPen = atributos_entrenamientoPen.values
objetivo_entrenamientoPen = objetivo_entrenamientoPen.values
atributos_pruebaPen = atributos_pruebaPen.values
objetivo_pruebaPen= objetivo_pruebaPen.values

print(atributos_entrenamientoPen)

[[ 97  98  19 ...   0  31  43]
 [ 53  93   3 ... 100   9  98]
 [100 100  57 ...  22  10   7]
 ...
 [ 57  94  39 ...  24  41   5]
 [  0  64  43 ...  23  74   0]
 [ 46  99   7 ...   0   0   6]]


***MODELOS PEN***

In [156]:

modelRandomForestPen = RandomForestClassifier(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestPen.fit(atributos_entrenamientoPen, objetivo_entrenamientoPen)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainPen = xgb.DMatrix(atributos_entrenamientoPen, label=objetivo_entrenamientoPen)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
dtestPen2 = xgb.DMatrix(atributos_pruebaPen)

# Definimos los parámetros del modelo XGBoost
paramsPen = {
    'objective': 'multi:softmax',  # Función objetivo para clasificación multiclase
    'num_class': 10,       # Número de clases en tu problema
    'eval_metric': 'merror' 
}

# Entrenamos el modelo XGBoost
modelXGBoostPen = xgb.train(paramsPen, dtrainPen)

In [157]:
clon_model_XGboost_pen = copy.deepcopy(modelXGBoostPen)
prediccionesPenXG = clon_model_XGboost_pen.predict(dtestPen)
explicacionesPenXG = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_pen, rangosPen)
    explicacionesPenXG.append(explicacion)
print(explicacionesPenXG)

[array([[-0.30920879, -0.08110741, -0.04974303, -0.20420814, -0.14184313,
        -0.10315301,  0.15720689, -0.10458417,  0.19483664,  0.19715444,
         0.11205435, -0.01721138,  0.05377255, -0.02440121, -0.20643562,
         0.52687104]]), array([[-0.15769884, -0.09916418, -0.09026083, -0.23966289, -0.13331438,
         0.17783198, -0.00256851, -0.08080821,  0.02799384,  0.12933998,
         0.02835886,  0.04001497, -0.0466639 ,  0.07191075,  0.16485727,
         0.2098341 ]]), array([[-0.06483321, -0.02030295, -0.021318  , -0.04796758, -0.04282331,
        -0.00779123,  0.24489172, -0.0267206 ,  0.03915151,  0.27948447,
        -0.21465126, -0.21114281, -0.04423322,  0.06776863, -0.22421788,
         0.29470571]]), array([[-0.13190767, -0.07284862, -0.0968768 , -0.22834274, -0.13319493,
         0.23468962, -0.05513768, -0.05461954, -0.04731688,  0.14376094,
        -0.01185326,  0.05007805,  0.07530773,  0.00906676,  0.07459462,
         0.2446004 ]]), array([[-0.23731581,  0.033

In [158]:
prediccionesPenRF = modelRandomForestPen.predict(atributos_pruebaPen)
explicacionesPenRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestPen, rangosPen)
    explicacionesPenRF.append(explicacion)
print(explicacionesPenRF)

[array([[-0.12964972, -0.05732269, -0.03224131, -0.09618108, -0.09922583,
        -0.00734106,  0.27871311, -0.03892117,  0.0797781 ,  0.31064921,
        -0.18099949, -0.20561486, -0.10035204,  0.1171738 , -0.18586293,
         0.34739796]]), array([[-0.39375138,  0.28102703, -0.06152724, -0.31489715,  0.24367688,
        -0.15130061,  0.04588506, -0.14925891, -0.08349882,  0.47201709,
        -0.07826571, -0.21711555,  0.06530375,  0.31460067, -0.14569009,
         0.17279498]]), array([[-0.51237543, -0.04773761, -0.13159368, -0.4112755 , -0.14229069,
        -0.13853594,  0.15427244, -0.00691181,  0.05504484,  0.20145288,
         0.31092255,  0.02545427, -0.12982172,  0.30823087,  0.29075094,
         0.17441361]]), array([[-0.53225762,  0.24246884, -0.10154448, -0.492918  ,  0.12374686,
        -0.08892213, -0.09628096, -0.05729985, -0.11840898,  0.40159209,
         0.13066553,  0.02211661,  0.09259087,  0.37014444, -0.02899836,
         0.13330514]]), array([[ 0.08956971, -0.018

In [159]:
nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoPen).iloc[2]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)
Iterac=10
atributosAcambiar=5


#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestPen))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostPen))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestPen,rangosPen))
#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostPen,rangosPen))

    0    1   2   3   4   5   6   7   8   9    10  11  12  13  14  15
2  100  100  57  91  22  62   0  31  18   1  100   0  85  22  10   7


***VALIDACIÓN***

In [160]:
evaluate_model(modelRandomForestPen, atributos_entrenamientoPen, objetivo_entrenamientoPen, atributos_pruebaPen, objetivo_pruebaPen)

Puntuación media de validación cruzada: 0.9814741035856573
Puntuación en el conjunto de prueba: 0.9919126566922766


In [161]:
def eliminar_irrelevantes_RF(pesos, k, muestra, modelo):
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    copia = muestra.copy()
    for i in range(len(muestra)):
        if i in indices:
            copia[i] = 0
    muestra_2d = copia.reshape(1, -1)
    prediccion_nueva = modelo.predict(muestra_2d)
    return prediccion_nueva


def eliminar_irrelevantes_XG(pesos, k, muestra, modelo):
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    
    data = muestra.get_data()
    data_copy = data.copy()
    label = muestra.get_label()
    weight = muestra.get_weight()
    base_margin = muestra.get_base_margin()

    # Copiar los valores y crear una nueva muestra
    
    for i in indices:
        data_copy[0,i]=0
    copia = xgb.DMatrix(data_copy, label=label, weight=weight, base_margin=base_margin)
    prediccion_nueva = modelo.predict(copia)
    return prediccion_nueva




In [162]:
predicciones_RF_Bicis = prediccionesBicisRF[:256]
objetivos_bicis = objetivo_pruebaBicis[:256]
explicaciones_RF_Bicis = explicacionesBicisRF

predicciones_XG_Bicis = prediccionesBicisXG[:256]
explicaciones_XG_Bicis = explicacionesBicisXG

predicciones_RF_Pen = prediccionesPenRF[:256]
objetivos_pen = objetivo_pruebaPen[:256]
explicaciones_RF_Pen = explicacionesPenRF

predicciones_XG_Pen = prediccionesPenXG[:256]
explicaciones_XG_Pen = explicacionesPenXG

In [163]:
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
predicciones_alteradas_RF_Bicis=[]
predicciones_alteradas_RF_Pen=[]
predicciones_alteradas_XG_Bicis=[]
predicciones_alteradas_XG_Pen=[]


for i in range(256):
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Bicis[i],5,atributos_pruebaBicis[i],modelRandomForestBicis)
    predicciones_alteradas_RF_Bicis.append(pred)
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Pen[i],5,atributos_pruebaPen[i],modelRandomForestPen)
    predicciones_alteradas_RF_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Pen[i],5,dtestPen2.slice([i]),modelXGBoostPen)
    predicciones_alteradas_XG_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Bicis[i],5,dtestBicis2.slice([i]),modelXGBoostBicis)
    predicciones_alteradas_XG_Bicis.append(pred)
print(len(predicciones_alteradas_XG_Bicis))
print(len(predicciones_alteradas_XG_Pen))

256
256


***MÉTRICAS***

In [164]:
# Identidad

def identidad(i, muestra, lista_explicaciones):
    e = lista_explicaciones[i]
    return np.linalg.norm(muestra - muestra) == 0 and np.linalg.norm(e - e) == 0

# Separabilidad

def separabilidad(i,j,muestra_i,muestra_j, lista_explicaciones):
    e1 = lista_explicaciones[i]
    e2 = lista_explicaciones[j]
    return np.linalg.norm(muestra_i - muestra_j) != 0 and np.linalg.norm(e1 - e2) > 0 

# Estabilidad

def estabilidad(i, muestra, lista_muestras, lista_explicaciones):
    e = lista_explicaciones[i]
    d_muestras = np.array([np.linalg.norm(muestra - x) for x in lista_muestras])
    d_explicaciones = np.array([np.linalg.norm(e - x) for x in lista_explicaciones])
    return spearmanr(d_muestras, d_explicaciones)[0] > 0

# Selectividad

def selectividad(i):
    return null

# Coherencia
def coherencia(i, lista_predicciones, lista_objetivo,predicciones_actualizadas):
    error_prediccion_original = lista_predicciones[i] - lista_objetivo[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - lista_objetivo[i]
    return np.abs(error_prediccion_original - error_prediccion_nuevo)

# Completitud

def completitud(i, lista_predicciones, lista_objetivo,predicciones_actualizadas):
    error_prediccion_original = lista_predicciones[i] - lista_objetivo[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - lista_objetivo[i]
    return error_prediccion_nuevo / error_prediccion_original

# Congruencia
def congruencia(lista_predicciones, lista_objetivo,predicciones_actualizadas):
    sumatorio_coherencia = 0
    sumatorio_desviacion = 0
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_coherencia += alpha
    coherencia_promedio = sumatorio_coherencia / len(lista_predicciones)
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_desviacion += np.power(alpha-coherencia_promedio,2)
    arg = sumatorio_desviacion / len(lista_predicciones)
    return np.sqrt(arg)

In [165]:
# Bicis Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaBicis[i],explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,atributos_pruebaBicis[i],atributos_pruebaBicis[i+1],explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaBicis[i],atributos_pruebaBicis[:256],explicaciones_RF_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Bicis,objetivos_bicis,predicciones_alteradas_RF_Bicis)
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_RF_Bicis,objetivos_bicis,predicciones_alteradas_RF_Bicis)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Bicis, objetivos_bicis,predicciones_alteradas_RF_Bicis))


Identidad: True

Separabilidad: True

Estabilidad: False

Coherencia: [23.87570313]

Completitud: [nan]

Congruencia: [50.62604698]


<ipython-input-164-1fd46aaf5ae7>:38: RuntimeWarning: invalid value encountered in divide
  return error_prediccion_nuevo / error_prediccion_original
<ipython-input-164-1fd46aaf5ae7>:38: RuntimeWarning: divide by zero encountered in divide
  return error_prediccion_nuevo / error_prediccion_original


In [166]:
# Bicis XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestBicis.slice([i]).get_data(),explicaciones_XG_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,dtestBicis.slice([i]).get_data(),dtestBicis.slice([i+1]).get_data,explicaciones_XG_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
lista_bicis = [x for x in dtestBicis[:256]]
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,dtestBicis.slice([i]).get_data(),lista_bicis,explicaciones_XG_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)
print("\nCoherencia:", res_coherencia/256)

# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Bicis, objetivos_bicis,predicciones_alteradas_XG_Bicis))

ValueError: dimension mismatch

In [ ]:
# Pen Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaPen[i],explicaciones_RF_Pen)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,atributos_pruebaPen[i],atributos_pruebaPen[i+1],explicaciones_RF_Pen)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaPen[i],atributos_pruebaPen[:256],explicaciones_RF_Pen)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Pen, objetivos_pen,predicciones_alteradas_RF_Pen))

In [ ]:
# Pen XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestPen.slice([i]).get_data(),explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, i+1,dtestPen.slice([i]).get_data(),dtestPen.slice([i+1]).get_data(),explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
lista_pen = [x for x in dtestPen[:256]]
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,dtestPen.slice([i]).get_data(),lista_pen,explicaciones_RF_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

res_selectividad = True

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)
print("\nCompletitud", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Pen, objetivos_pen,predicciones_alteradas_XG_Pen))